In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
df = pd.read_csv('C://Users//jack//Documents//stock_database_pipeline//price_prediction_pipeline//dataset//update//update_pre_cleantestRouters.csv')

df.head()

In [ ]:
# Drop all columns except , 'price', 'title', 'link', 'condition', 'brand', 'type', 'seller_notes'
df = df[['price', 'title', 'link', 'condition', 'brand', 'type', 'seller notes']]

In [ ]:
# List all unique values in the brand and type columns
print(df['brand'].unique())

df['type'].unique()

In [ ]:
source = pd.read_csv('C://Users//jack\Documents//stock_database_pipeline//price_prediction_pipeline//source_csv//merged_inventory.csv')

In [ ]:
# Filter source for 'server' in the 'product type' column
source = source[source['ProductType'] == 'router']

In [ ]:
# Create a regex pattern to match any model in the source['Model'] list
pattern = '|'.join(re.escape(device) for device in source['Model'])

# Extract the model from the title using the regex pattern
df['model'] = df['title'].str.extract(f'({pattern})', expand=False)

In [ ]:
# Create a regex pattern to match any model in the source['Manufacturer'] list
pattern = '|'.join(re.escape(device) for device in source['Manufacturer'])

# Extract the model from the title using the regex pattern
df['brand'] = df['title'].str.extract(f'({pattern})', expand=False)

In [ ]:
# Reorder the columns
df = df[['price', 'brand', 'model', 'title', 'type', 'link', 'condition', 'seller notes']]

df.head()

In [ ]:
# Drop rows where pattern is present in the title
pattern = r'shower|wipers|wiper|joblot|job|x4|laptop|thinkpad|rails|kit|cable|cables|screws|screw|mount|mounts|mounting|bracket|brackets|bezel|bezels|cover|covers|tray|trays|sled|sleds|psu|power supply|power supplies|fan|fans|caddy|caddies|heatsink'

# Drop NaN values in the title column
df = df.dropna(subset=['title'])

# Drop rows where pattern is present in the title
df = df[~df['title'].str.contains(pattern, case=False)]

In [ ]:
# Drop all rows missing a model
df = df.dropna(subset=['model'])

df = df[['price', 'brand', 'model', 'title', 'type', 'link', 'condition', 'seller notes']]


print(len(df))

In [ ]:
df['model'].unique()

In [ ]:
# Save the cleaned data to a new csv file
df.to_csv('C://Users//jack//Documents//stock_database_pipeline//price_prediction_pipeline//dataset//update//update_cleantestRouters.csv', index=False)

In [ ]:
df['status'] = None

In [ ]:
# Reorder columns
df = df[['status', 'price', 'brand', 'model', 'title', 'type', 'link', 'condition', 'seller notes']]

df.head()

In [ ]:
# Group by 'model' and 'brand', then calculate the average price and keep the status
unique_df = df.groupby(['brand', 'model']).agg(
    status=('status', 'first'),  # Assuming status is the same within each group
    average_price=('price', 'mean')
).reset_index()

print(unique_df)

unique_df['status'] = unique_df['average_price'].astype(float).apply(lambda x: 'sell' if x >= 50 else 'scrap')
print(unique_df)
# Drop 'avereage_price' column
unique_df = unique_df.drop(columns=['average_price'])

# Rename columns
unique_df.columns = ['Manufacturer', 'Model', 'Status']

In [ ]:
# Capitalize the first letter of each word in 'Manufacturer' and 'Model' columns
unique_df['Manufacturer'] = unique_df['Manufacturer'].str.title()
unique_df['Model'] = unique_df['Model'].str.title()
unique_df['Status'] = unique_df['Status'].str.title()

# Merge all three columns into one separated by a ; character
unique_df['Manufacturer;Model;Status'] =unique_df['Manufacturer'] + ';' +unique_df['Model'] + ';' +unique_df['Status']

# Drop the 'Manufacturer' and 'Model' columns
unique_df = unique_df.drop(columns=['Manufacturer', 'Model', 'Status'])

unique_df.head()

# Save the cleaned data to a new csv file
unique_df.to_csv('C://Users//jack//Documents//stock_database_pipeline//price_prediction_pipeline//source_csv//glpi_update_router_data.csv', index=False)